In [1]:
import pymc as pm
import numpy as np
import arviz as az
import pandas as pd

%load_ext lab_black
%load_ext watermark

# Wine classification

This example demonstrates ...

Adapted from [unit 10: italywines123.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/italywines123.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/r.txt).

## Associated lecture video: Unit 10 Lesson 5

In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=102" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

## Problem statement

This popular data set was provided by Forina et al (1988). 
The data below consist of results of a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents found in each of the three types of wines. 

  The variables are
  
   Column   Variable   Description  
   
  ==========================
  
      1       Y     Type (Response, 1,2,3)  
                      1 == 1 0 0;    2 == 0 1 0;   3 == 0 0 1
                      
      2      X1    Alcohol 
      
 	 3      X2    Malic acid 
     
 	 4      X3    Ash 
     
	  5      X4    Alcalinity of ash 
      
  	6      X5    Magnesium 
    
	  7      X6    Total phenols 
      
 	 8      X7    Flavanoids 
     
      9      X8    Nonflavanoid phenols 
      
     10     X9    Proanthocyanins 
     
     11     X10  Color intensity 
     
     12     X11  Hue 
     
     13     X12 OD280/OD315 of diluted wines 
     
     14     X13  Proline 
 



(a) Fit the multinomial regression
that predicts the type of wine Y from predictors  X1 - X13.
What are estimated coefficients? What is the deviance?

(b) What is your prediction for  pp=P(Ynew=1)  if a new case has attributes  
 XX1 = 12.9, XX2 = 2,  XX3 = 2.4, XX4 = 17,
 XX5 = 100,  XX6 = 2.8, XX7 = 2.1, XX8 =  0.35,  XX9 = 1.6, XX10 = 5, 
XX11 = 1.05,  XX12 = 3, and XX13 = 750?
How would you classify this wine type, as 1, 2, or 3?

 
 Forina, M.,   Leardi, R.,   Armanino, C., and  Lanteri, S. (1988).
 PARVUS: An extendable package of programs for data exploration, classification and correlation,  Elsevier, Amsterdam,   ISBN 0-444-43012-1;
 also, Report at Institute of Pharmaceutical and Food Analysis and Technologies, Via Brigata Salerno,
  16147 Genoa, Italy.